In [6]:
import bs4
import requests
import pandas as pd

In [7]:

def scrape_listings(url):
    rows = []
    r = requests.get(url)

    soup = bs4.BeautifulSoup(r.text, 'html.parser')

    listings = soup.find_all('div', {'class': 'result-item'})
    for element in listings:
        row = {}
        row['url'] = element.find('a').get('href')

        url = 'https://www.exchangeandmart.co.uk' + row['url']
        det_url = requests.get(url)
        soup = bs4.BeautifulSoup(det_url.text, 'html.parser')
        name = soup.find('span', {'class': 'ttl'})
        model = soup.find('span', {'class': 'ttlSup'})
        price = soup.find('span', {'itemprop': 'price'})
        row['name'] = str(name.text).strip()
        row['model'] = str(model.text).strip()
        row['price'] = str(price.text).strip()

        facts = element.find('div', {'class': 'key-details'})
        sayi = 1
        for fact in facts.find_all('p'):
            if(sayi != 4 and sayi != 5):
                row[sayi] = str(fact.text).strip()
            sayi+=1
            #add model and name to rows
        
        det1 = soup.find('div', {'class': 'adDetails'})
        det2 = soup.find('div', {'class': 'adDetsColumn'})

        for cell in det1.find_all('div', {'class': 'adDetsItem'}):
            row[sayi] = str(cell.text).strip()
            sayi+=1
        counter = 1
        
        for span in det2.find_all('span'):
            if counter%2 == 0:
                row[sayi] = str(span.text).strip()
                sayi+=1
            counter        

        rows.append(row)

    return pd.DataFrame(rows)


In [8]:
# Solution
df = scrape_listings('https://www.exchangeandmart.co.uk/used-cars-for-sale/toyota/any-distance-from-wc1e-6dp/fuel-hybrid/page')
df

,url,name,model,price,1,2,3,6,7,8,9,10,11,12,13,14
0,/ad/30919027,Toyota RAV4,4 2.5 VVT-i Hybrid Excel 5dr CVT SUV,"28,179",2019,Automatic,49.6 mpg,Hybrid,2019,2.5L,"11,923 miles",Hybrid,Automatic,Grey,SUV,49.6 mpg
1,/ad/30854130,Toyota Yaris,s 1.5 Hybrid Icon Tech 5dr CVT Hatchback,"15,250",2019,Automatic,76.3 mpg,Hybrid,2019,1.5L,"12,868 miles",Hybrid,Automatic,Other,Hatchback,76.3 mpg
2,/ad/30830309,Toyota Yaris,1.5 Hybrid Excel 5dr CVT [15 inch] Hatchback,"15,899",2019,Other,67.3 mpg,Hybrid,2019,1497cc,"12,161 miles",Hybrid,N/A,Red,Hatchback,67.3 mpg
3,/ad/30664215,Toyota C-HR,1.8 Hybrid Icon 5dr CVT [Tech] SUV,"18,799",2021,Other,57.7 mpg,Hybrid,2021,1798cc,"14,537 miles",Hybrid,N/A,White,SUV,57.7 mpg
4,/ad/30845537,Toyota C-HR,1.8 Hybrid Dynamic 5dr CVT SUV,"19,849",2019,Other,72.4 mpg,Hybrid,2019,1798cc,"17,250 miles",Hybrid,N/A,Blue,SUV,72.4 mpg
5,/ad/30643432,Toyota Corolla,2.0 VVT-i Hybrid Design 5dr CVT Hatchback,"18,599",2020,Other,72.4 mpg,Hybrid,2020,1987cc,"17,946 miles",Hybrid,N/A,Black,Hatchback,72.4 mpg
6,/ad/30438194,Toyota Corolla,1.8 VVT-i Hybrid Icon Tech 5dr CVT Hatchback,"15,199",2020,Other,62.8 mpg,Hybrid,2020,1798cc,"18,099 miles",Hybrid,N/A,White,Hatchback,62.8 mpg
7,/ad/30719667,Toyota Corolla,1.8 VVT-i Hybrid Icon Tech 5dr CVT Hatchback,"16,599",2020,Other,62.8 mpg,Hybrid,2020,1798cc,"18,272 miles",Hybrid,N/A,Black,Hatchback,62.8 mpg
8,/ad/30927887,Toyota Yaris,1.5 Hybrid Icon 5dr CVT Hatchback,"12,849",2019,Other,Tax: £170,2019,1497cc,"19,110 miles",Hybrid,N/A,Red,Hatchback,NaN,NaN
9,/ad/30770779,Toyota C-HR,1.8 Hybrid Icon 5dr CVT SUV,"17,999",2020,Other,58.9 mpg,Hybrid,2020,1798cc,"19,339 miles",Hybrid,N/A,Blue,SUV,58.9 mpg


In [9]:
def scrape_all(pages):
    url = 'https://www.exchangeandmart.co.uk/used-cars-for-sale/toyota/fuel-petrol'
    df = pd.DataFrame()
    for page in pages:
        df = pd.concat([df, scrape_listings(url + str(page))], ignore_index=True)
    return df

In [10]:
df_large = scrape_all(range(1, 4))
df_large

,url,name,model,price,1,2,3,6,7,8,9,10,11,12,13,14
0,/ad/30788471,Toyota Aygo,o X 1.0 VVT-i Edge 5dr Hatchback,"16,750",2023,Manual,58.9 mpg,Petrol,2023,1L,2 miles,Petrol,Manual,Beige,Hatchback,58.9 mpg
1,/ad/30701735,Toyota Aygo,X 1.0 VVT-i Edge 5dr Hatchback,"16,250",2023,Manual,58.9 mpg,Petrol,2023,998cc,3 miles,Petrol,Manual,Red,Hatchback,58.9 mpg
2,/ad/30924791,Toyota Aygo,o X 1.0 VVT-i Edge 5dr Hatchback,"17,290",2023,Manual,58.9 mpg,Petrol,2023,1L,3 miles,Petrol,Manual,Red,Hatchback,58.9 mpg
3,/ad/30701729,Toyota Aygo,X 1.0 VVT-i Undercover 5dr Auto Hatchback,"19,650",2023,Automatic,56.5 mpg,Petrol,2023,998cc,3 miles,Petrol,Automatic,Grey,Hatchback,56.5 mpg
4,/ad/30701733,Toyota Corolla,1.8 Hybrid Design 5dr CVT Hatchback,"28,550",2023,Automatic,61.4 mpg,Hybrid,2023,1798cc,3 miles,Petrol Hybrid,Automatic,White,Hatchback,61.4 mpg
5,/ad/30778327,Toyota C-HR,2.0 Hybrid GR Sport 5dr CVT SUV,"36,500",2023,Automatic,53.3 mpg,Hybrid,2023,1987cc,3 miles,Petrol Hybrid,Automatic,Unknown,SUV,53.3 mpg
6,/ad/30837459,Toyota Aygo,o X 1.0 VVT-i Edge 5dr Hatchback,"17,133",2023,Manual,58.9 mpg,Petrol,2023,1L,4 miles,Petrol,Manual,Blue,Hatchback,58.9 mpg
7,/ad/30922212,Toyota Yaris,1.5 Hybrid Design 5dr CVT (Nav) Hatchback,"23,473",2023,Automatic,68.9 mpg,Hybrid,2023,1490cc,4 miles,Petrol Hybrid,Automatic,Blue,Hatchback,68.9 mpg
8,/ad/30922210,Toyota Yaris,1.5 Hybrid Design 5dr CVT (Nav) Hatchback,"23,473",2023,Automatic,68.9 mpg,Hybrid,2023,1490cc,4 miles,Petrol Hybrid,Automatic,Blue,Hatchback,68.9 mpg
9,/ad/30929826,Toyota Yaris,1.5 Hybrid Design 5dr CVT (Nav) Hatchback,"23,473",2023,Automatic,68.9 mpg,Hybrid,2023,1490cc,4 miles,Petrol Hybrid,Automatic,Red,Hatchback,68.9 mpg


In [11]:
df_large.to_csv('file_name.csv', index=False)